<a href="https://colab.research.google.com/github/Songhyunjeong/AutoRAG-tutorial-ko/blob/main/AutoRAG_Tutorial_3_Use_Custom_LLM_%26_Embedding_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

## Install and setting AutoRAG

In [ ]:
%%shell
apt-get remove python3-blinker
pip install blinker==1.8.2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'python3-blinker' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-glib-2.0 gir1.2-packagekitglib-1.0 libappstream4 libgirepository-1.0-1
  libglib2.0-bin libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0 libstemmer0d
  libxmlb2 libyaml-0-2 lsb-release packagekit pkexec policykit-1 polkitd python-apt-common
  python3-apt python3-cffi-backend python3-cryptography python3-dbus python3-distro python3-gi
  python3-httplib2 python3-importlib-metadata python3-jeepney python3-jwt python3-keyring
  python3-lazr.uri python3-more-itertools python3-pkg-resources python3-pyparsing
  python3-secretstorage python3-six python3-wadllib python3-zipp
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
%pip install -Uq ipykernel==5.5.6 ipywidgets-bokeh==1.0.2

In [ ]:
%pip install -Uq AutoRAG==0.3.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.3/234.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Custom LLM & embedding model

You can use most of the LLM and embedding models from LlamaIndex in AutoRAG.
However, you have to add some snippet of code to use it.
It is not that hard.
Let's find out how can add your own LLM model or embedding model in AutoRAG.

## AutoRAG default supporting models

Since you can use any LlamaIndex models in AutoRAG, but AutoRAG supports some models as default without your own configuration.

### Default supported LLMs

| LLM Model Type | llm parameter  |
|:--------------:|:--------------:|
|     OpenAI     |     openai     |
| HuggingFaceLLM | huggingfacellm |
|   OpenAILike   |   openailike   |
|     Ollama     |     ollama     |


### Default supported Embedding Models


|                                           Embedding Model Type                                            |       embedding_model parameter       |
|:---------------------------------------------------------------------------------------------------------:|:-------------------------------------:|
|                             Default openai embedding (text-embedding-ada-002)                             |                openai                 |
|                              openai large embedding (text-embedding-3-large)                              |         openai_embed_3_large          |
|                              openai small embedding (text-embedding-3-small)                              |         openai_embed_3_small          |
|                  [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5)                  |      huggingface_baai_bge_small       |
|               [cointegrated/rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny2)               | huggingface_cointegrated_rubert_tiny2 |
| [sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) |     huggingface_all_mpnet_base_v2     |
|                             [BAAI/bge-m3](https://huggingface.co/BAAI/bge-m3)                             |          huggingface_bge_m3           |



## Configure custom LLM model

Beside default LLM model, you can use other LLM model in LlamaIndex.
You can check supported LlamaIndex LLM list at [here](https://docs.llamaindex.ai/en/stable/module_guides/models/llms/modules/).

In this tutorial, we will use Upstage Model at AutoRAG. So, for using it, you must prepare Upstage API key at [here](https://console.upstage.ai/home).

And, you have to add your Upstage api key at secrets. Check out left side and go to 'Secrets' tab. At there, press 'add new secret' and set name to UPSTAGE_API_KEY. And set value to your openai api key. Be sure to press toggle for notebook access!

You can check out the usage of the Upstage model in LlamaIndex at [here](https://docs.llamaindex.ai/en/stable/examples/llm/upstage/).

In [ ]:
from google.colab import userdata
import os
upstage_api_key = userdata.get('UPSTAGE_API_KEY')
assert bool(upstage_api_key), "You have to set OPENAI_API_KEY at colab secrets."
os.environ["UPSTAGE_API_KEY"] = upstage_api_key

After configuring UPSTAGE_API_KEY, let's add Upstage LLM class to AutoRAG.

For adding this, you have to install LlamaIndex Upstage extension first.

In [ ]:
%pip install llama-index-llms-upstage

In [ ]:
import autorag

from llama_index.llms.upstage import Upstage

autorag.generator_models['upstage'] = Upstage

## Configure custom embedding model

After LLM, let's add Upstage embedding model to AutoRAG. It is simple also.

In [ ]:
%pip install llama-index-embeddings-upstage

In [ ]:
import autorag
from autorag import LazyInit
from llama_index.embeddings.upstage import UpstageEmbedding

autorag.embedding_models['upstage'] = LazyInit(UpstageEmbedding, model_name="solar-embedding-1-large")

## Let's use custom LLM & embedding model!

Now configuration is over.
For using custom models in AutoRAG, you can't use cli command.
Instead, you can use python code easily.
It is not hard.

If you skip the first tutorial of AutoRAG, see [here](https://colab.research.google.com/drive/19OEQXO_pHN6gnn2WdfPd4hjnS-4GurVd?usp=sharing).

In [ ]:
import os
os.makedirs('/content/eli5_data')

FileExistsError: [Errno 17] File exists: '/content/eli5_data'

In [ ]:
import pathlib

from datasets import load_dataset

def load_eli5_dataset(save_path):
    # set file path
    file_path = "MarkrAI/eli5_sample_autorag"

    # load dataset
    corpus_dataset = load_dataset(file_path, "corpus")['train'].to_pandas()
    qa_train_dataset = load_dataset(file_path, "qa")['train'].to_pandas()
    qa_test_dataset = load_dataset(file_path, "qa")['test'].to_pandas()

    # save data
    if os.path.exists(os.path.join(save_path, "corpus.parquet")) is True:
        raise ValueError("corpus.parquet already exists")
    if os.path.exists(os.path.join(save_path, "qa.parquet")) is True:
        raise ValueError("qa.parquet already exists")
    corpus_dataset.to_parquet(os.path.join(save_path, "corpus.parquet"))
    qa_train_dataset.to_parquet(os.path.join(save_path, "qa_train.parquet"))
    qa_test_dataset.to_parquet(os.path.join(save_path, "qa_test.parquet"))

load_eli5_dataset("/content/eli5_data")

Some datasets params were ignored: ['splits']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.
Some datasets params were ignored: ['splits']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.
Some datasets params were ignored: ['splits']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


ValueError: corpus.parquet already exists

In [ ]:
import pandas as pd
qa_df = pd.read_parquet('/content/eli5_data/qa_train.parquet')
sample_qa_df = qa_df.sample(20, random_state=42) # In this sample code, we will only optimize pipeline with 20 samples just for testing.
sample_qa_df.reset_index(drop=True, inplace=True)
sample_qa_df.to_parquet('/content/eli5_data/qa_sample.parquet')

In [ ]:
from itertools import chain
from autorag.utils.util import to_list
# We drop unused corpus dataframe for faster inference.
corpus_df = pd.read_parquet('/content/eli5_data/corpus.parquet')
target_retrieval_gts = list(chain.from_iterable(to_list(sample_qa_df["retrieval_gt"].tolist())))
target_retrieval_gts = list(chain.from_iterable(target_retrieval_gts))
sample_corpus_df = corpus_df[corpus_df["doc_id"].isin(target_retrieval_gts)]
sample_corpus_df.reset_index(drop=True, inplace=True)
sample_corpus_df.to_parquet('/content/eli5_data/corpus_sample.parquet')

In [ ]:
len(sample_corpus_df)

20

The last thing for using custom models, you can just write in YAML file now.
You can find `upstage` at `embedding_model` and `llm` setting at `generator`.

In [ ]:
%%writefile config.yaml
vectordb:
  - name: chroma_upstage
    db_type: chroma
    client_type: persistent
    collection_name: upstage
    embedding_model: upstage
    path: ${PROJECT_DIR}/resources/chroma
node_lines:
- node_line_name: retrieve_node_line
  nodes:
    - node_type: retrieval
      strategy:
        metrics: [retrieval_f1, retrieval_recall, retrieval_ndcg, retrieval_mrr]
      top_k: 3
      modules:
        - module_type: vectordb
          vectordb: chroma_upstage
        - module_type: bm25
        - module_type: hybrid_rrf
          weight_range: (4,80)
- node_line_name: post_retrieve_node_line
  nodes:
    - node_type: prompt_maker
      strategy:
        metrics:
          - metric_name: meteor
          - metric_name: rouge
          - metric_name: sem_score
            embedding_model: upstage # Use upstage embedding model
      modules:
        - module_type: fstring
          prompt: "Read the passages and answer the given question. \n Question: {query} \n Passage: {retrieved_contents} \n Answer : "
    - node_type: generator
      strategy:
        metrics:
          - metric_name: meteor
          - metric_name: rouge
          - metric_name: sem_score
            embedding_model: upstage # Use upstage embedding model
      modules:
        - module_type: llama_index_llm
          llm: upstage  # Use upstage LLM
          batch: 4

Overwriting config.yaml


In [ ]:
%rm -rf /content/project_dir

In [ ]:
# make project folder
import os
os.makedirs('/content/project_dir')

In [ ]:
from autorag.evaluator import Evaluator
evaluator = Evaluator(qa_data_path='/content/eli5_data/qa_sample.parquet', corpus_data_path='/content/eli5_data/corpus_sample.parquet',
                      project_dir='/content/project_dir')

In [ ]:
evaluator.start_trial('/content/config.yaml', skip_validation=True)

Output()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: 
`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in
transformers v4.45, and will be then set to `False` by default. For more details check this issue: 
https://github.com/huggingface/transformers/issues/31884
  warnings.warn(

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Generating embeddings:   0%|          | 0/20 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/20 [00:00<?, ?it/s]

# [Tip] Run Local Models

To run local models, we recommend to use vllm for faster inference.
`vllm` module is optimized for AutoRAG system.
To learn how to run it, you can go to [here](https://docs.auto-rag.com/nodes/generator/vllm.html) for checking documentation.